In [2]:
## install needed modules
! pip install geopandas
! pip install fulcrum
! pip install matplotlib
! pip install fiona

   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   -- ------------------------------------- 0.8/12.6 MB 4.2 MB/s eta 0:00:03
   ------------ --------------------------- 3.9/12.6 MB 9.4 MB/s eta 0:00:01
   -------------- ------------------------- 4.7/12.6 MB 8.6 MB/s eta 0:00:01
   ------------------------------- -------- 10.0/12.6 MB 11.9 MB/s eta 0:00:01
   ---------------------------------------- 12.6/12.6 MB 12.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
   ------------------- -------------------- 5.5/11.5 MB 28.0 MB/s eta 0:00:01
   -------------------------------------- - 11.0/11.5 MB 26.5 MB/s eta 0:00:01
   ---------------------------------------- 11.5/11.5 MB 23.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/16.2 MB ? eta -:--:--
   ------------ --------------------------- 5.0/16.2 MB 27.4 MB/s eta 0:00:01
   ------------------------- -------------- 10.2/16.2 MB 27.7 MB/s eta 0:00:01
   -----

In [3]:
## import modules
import math, json
from collections import Counter
from datetime import datetime, timedelta
from geopandas.tools import sjoin
# from pyproj import proj
import pandas as pd
import csv
import numpy as np
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
import matplotlib.pyplot as plt
from shapely.geometry import Point
import geopandas as gpd
import fiona
from fulcrum import Fulcrum

In [4]:
## variables
apiToken = "689a52b9420360bef6af63ad889bcbf4cb34f26e1d1348b2d3a2ff3b71dfe6828d07395724415aa9"
urlBase = 'https://api.fulcrumapp.com/api/v2/'
fulcrum = Fulcrum(key=apiToken)
osTimestamp = datetime.today() - timedelta(days=4)
recordsPerPage = 5000
print(fulcrum)

In [5]:
## data I
appID = "0092aa7f-a2e8-4972-8f26-46eb98b23a12" # ID of Combined Tree Invnetory
formdataSource = fulcrum.forms.find(appID)
# get number of pages
## the following divides the total number of records in chunks and downloads and assembles them. Otherwise the fulcrum API allows only downloads of 20,000 records

## count records in app
recordCountSource = fulcrum.forms.find(appID)['form']['record_count']
pagesSource = math.ceil(recordCountSource / recordsPerPage)

## get data I
if pagesSource > 1:
    for p in range(1, pagesSource + 1):
        dataPage = fulcrum.records.search(url_params={'form_id': appID, 'page': p, 'per_page': recordsPerPage})[
            'records']
        if p > 1:
            dataSource.extend(dataPage)
        else:
            dataSource = dataPage
else:
    dataSource = fulcrum.records.search(url_params={'form_id': appID, 'page': 1, 'per_page': recordsPerPage})['records']

In [6]:
## What do we have? Attributes:
type(dataSource)
cticolumns = list(dataSource[0].keys())
cticolumns

['status',
 'version',
 'id',
 'created_at',
 'updated_at',
 'client_created_at',
 'client_updated_at',
 'created_by',
 'created_by_id',
 'updated_by',
 'updated_by_id',
 'created_location',
 'updated_location',
 'created_duration',
 'updated_duration',
 'edited_duration',
 'form_id',
 'project_id',
 'record_series_id',
 'assigned_to',
 'assigned_to_id',
 'form_values',
 'latitude',
 'longitude',
 'altitude',
 'geometry',
 'speed',
 'course',
 'horizontal_accuracy',
 'vertical_accuracy',
 'system_status']

In [7]:
print(dataSource[0]["status"])
status_values = {record["status"] for record in dataSource if "status" in record}
print(status_values)

No action required - لا حاجة لأي إجراء
{'Request of inspection (RFI) - طلب معاينة', 'Approval of RFI - موافقة على المعاينة', 'Final on hold - نهائي عالق', 'Escalation issue - قضية تصعيد', 'Work order - أمر عمل', 'No action required - لا حاجة لأي إجراء', 'Final approval - موافقة نهائية', 'On hold - قضية عالقة', 'Reports', 'Issue in place - قضية موجودة', 'Rejection of RFI - رفض معاينة', 'Request of activation - طلب تفعيل'}


In [20]:
## data II
dqID = "0046e79b-ef91-4b37-99b9-d6340e28094c" # DQ ID
dqformdataSource = fulcrum.forms.find(dqID)
# get number of pages
## the following divides the total number of records in chunks and downloads and assembles them. Otherwise the fulcrum API allows only downloads of 20,000 records

## count records in app
dqrecordCountSource = fulcrum.forms.find(dqID)['form']['record_count']
dqpagesSource = math.ceil(dqrecordCountSource / recordsPerPage)

## get data II
if dqpagesSource > 1:
    for p in range(1, dqpagesSource + 1):
        dataPage = fulcrum.records.search(url_params={'form_id': dqID, 'page': p, 'per_page': recordsPerPage})[
            'records']
        if p > 1:
            dqdataSource.extend(dataPage)
        else:
            dqdataSource = dataPage
else:
    dqdataSource = fulcrum.records.search(url_params={'form_id': dqID, 'page': 1, 'per_page': recordsPerPage})['records']

In [21]:
dqdataSource[0]["status"]
dqstatus_values = {record["status"] for record in dqdataSource if "status" in record}
print(dqstatus_values)

{'No Action Required - نجز'}


In [17]:
for record in dqdataSource:
    record["status"] = 'No action required - لا حاجة لأي إجراء' 

In [ ]:
# Define the correct project ID
PROJECT_ID = '0092aa7f-a2e8-4972-8f26-46eb98b23a12' # 0092aa7f-a2e8-4972-8f26-46eb98b23a12

for record in dqdataSource:
    record_id = record.get("id")
    
    # Remove unnecessary fields 
    del(record["id"])
    del(record["version"])

    # Ensure the record has the correct project ID
    record["form_id"] = PROJECT_ID  

    # Upload record to Fulcrum
    CREATED = fulcrum.records.create(record)
    # Check CREATED
    #if CREATED and CREATED.get("form_id") == PROJECT_ID:
    #    print(f"✅ Record {record_id} successfully created under Project ID: {PROJECT_ID}")
    #else:
    #    print(f"❌ Failed to create Record {record_id} under Project ID: {PROJECT_ID}")

In [11]:
CREATED

{'record': {'errors': {'status': ["'No Action Required - نجز' is not a valid status"]}}}

In [18]:
dqcolumns = list(dqdataSource[0].keys())
dqcolumns == cticolumns

False

In [10]:
print(dqdataSource[0]["id"])
print(dqdataSource[1]["id"])
print(dqdataSource[0]["project_id"])
print(dqdataSource[1]["project_id"])

4eb7ba28-9b08-408c-97d0-f5ea7005431c
1e5fa24f-a9e0-414d-9876-c1eb384a40fc
None
None


In [ ]:
## Duplicated trees?
ICTres = list(set(record["id"] for record in dataSource))
DQres = list(set(record["id"] for record in dqdataSource))
common_trees = list(set(ICTres) & set(DQres)) or [0]
common_trees

In [10]:
print(record["form_id"])

0092aa7f-a2e8-4972-8f26-46eb98b23a12


In [49]:
## Merge DQ data to CTI data (Version II) 
merged_dataSource = dataSource + dqdataSource
print(len(merged_dataSource))
print(len(dataSource)+len(dqdataSource))
mcolumns = list(merged_dataSource[0].keys())
print(mcolumns == dqcolumns)

302787
302787
True


In [ ]:
# Send the updated dataSource (which now includes dqdataSource) to the server (Version II ~ 02)
import logging

logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

for record in merged_dataSource:
    try:
        # Send the record to the server
        response = fulcrum.records.create(record)
        logging.info(f"Record {record['id']} sent to server successfully.")
    except Exception as e:
        logging.error(f"Error sending record {record['id']} to server: {e}")